In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
from funciones_data_visualization import *
from variables_data_visualization import *
from pylab import rcParams
import geopandas as gpd
import contextily as ctx
import folium

In [2]:
boards_total_europe = pd.read_csv('women_boards_europe_total.csv')

In [3]:
boards_total_europe = boards_total_europe.iloc[2:]
boards_total_europe = boards_total_europe.drop(30)
boards_total_europe.rename(columns = {'Unnamed: 0':'Country', 'Total' : 'Total number'}, inplace = True)

In [4]:
boards_total_europe = boards_total_europe[['Country', 'Women.1']]

In [5]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [6]:
boards_total_europe = world.merge(boards_total_europe, how="left", left_on=['name'], right_on=['Country'])
print(boards_total_europe.head())

     pop_est      continent                      name iso_a3  gdp_md_est  \
0     920938        Oceania                      Fiji    FJI      8374.0   
1   53950935         Africa                  Tanzania    TZA    150600.0   
2     603253         Africa                 W. Sahara    ESH       906.5   
3   35623680  North America                    Canada    CAN   1674000.0   
4  326625791  North America  United States of America    USA  18560000.0   

                                            geometry Country  Women.1  
0  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...     NaN      NaN  
1  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...     NaN      NaN  
2  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...     NaN      NaN  
3  MULTIPOLYGON (((-122.84000 49.00000, -122.9742...     NaN      NaN  
4  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...     NaN      NaN  


In [7]:
table = boards_total_europe.dropna(subset=['Women.1'])

In [9]:
mapa_europa = folium.Map()

In [11]:
folium.Choropleth(
    geo_data=table,
    name='choropleth',
    data=table,
    columns=['Country', 'Women.1'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Percentage of women in decision making'
).add_to(mapa_europa)
mapa_europa.save('mapa_europa_porcentajes.html')

In [12]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    table,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Country', 'Women.1'],
        aliases=['Pais: ','Women percentage: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
mapa_europa.add_child(NIL)
mapa_europa.keep_in_front(NIL)
folium.LayerControl().add_to(mapa_europa)
mapa_europa.save('mapa_europa_porcentajes.html')

In [13]:
mapa_europa